In [280]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, Markdown
import base64
from PIL import Image
import io
import json
import subprocess

In [281]:
def get_video_entities(offset=0, limit=10):
    endpoint = "http://localhost:7200/repositories/mmkg"
    sparql = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX mssn: <http://mssn.sigappfr.org/mssn/>
    PREFIX vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/>
    PREFIX ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/>
    PREFIX ac: <http://kgrc4si.home.kg/virtualhome2kg/ontology/action/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    select distinct ?activity ?event ?prevObjectName ?action ?object ?frame ?targetObject
    where {
        ?prevEvent vh2kg:hasVideoSegment ?visualSegment .
        ?visualSegment mssn:hasMediaDescriptor ?frame_r ;
            vh2kg:hasEndFrame ?endFrame .
        ?frame_r vh2kg:frameNumber ?frame .
        filter(?frame < ?endFrame && ?frame >= (?endFrame - 5))
        ?frame_r mssn:hasMediaDescriptor ?bbox2d_1 .
        ?bbox2d_1 vh2kg:is2DbboxOf ?prevObject ;
                rdfs:label ?prevObjectName .
        SERVICE <https://kgrc4si.home.kg:7200/repositories/KGRC4SIv05_affordance> {
            ?activity vh2kg:hasEvent ?event .
            ?prevEvent vh2kg:action ac:grab ;
                vh2kg:mainObject ?prevObject ;
                vh2kg:nextEvent ?event .
            ?event vh2kg:action ?action ;
                vh2kg:mainObject ?object .
            optional { ?event vh2kg:targetObject ?targetObject }
        }
        BIND(RAND() AS ?rand)
    } order by ?rand offset """ + str(offset) +  """ limit """ + str(limit) + """
    """
    wrapper = SPARQLWrapper(endpoint)
    wrapper.setQuery(sparql)
    wrapper.setReturnFormat(JSON)
    results = wrapper.query().convert()
    return results

In [282]:
def get_video_base64(activity_ent):
    endpoint = "http://localhost:7200/repositories/mmkg"
    sparql = """
        PREFIX vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/>
        PREFIX sosa: <http://www.w3.org/ns/sosa/>
        PREFIX ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/>
        SELECT ?base64 ?frameRate WHERE {
            ?video vh2kg:isVideoOf <""" + activity_ent + """> ;
        sosa:madeBySensor ?camera .
    ?video vh2kg:video ?base64 ;
           vh2kg:frameRate ?frameRate .
    filter(?camera != ex:camera2_scene1 && ?camera != ex:camera2_scene2 && ?camera != ex:camera2_scene3 && ?camera != ex:camera2_scene4 && ?camera != ex:camera2_scene5 && ?camera != ex:camera2_scene6 && ?camera != ex:camera2_scene7)
        } order by ?base64 limit 1
    """
    wrapper = SPARQLWrapper(endpoint)
    wrapper.setQuery(sparql)
    wrapper.setReturnFormat(JSON)
    results = wrapper.query().convert()
    return results

In [283]:
import re
def remove_url_and_id(object_ent):
    object_name = object_ent.replace("http://kgrc4si.home.kg/virtualhome2kg/instance/", "")
    object_name = re.sub(r'\d+_scene\d', '', object_name)
    return object_name

In [284]:
few_shot_size = 10
test_size = 100

results = get_video_entities(offset=0, limit=few_shot_size+test_size)

In [285]:

bindings = results["results"]["bindings"]
image_list = []

few_shot_samples = []
ground_truth = []
i = 0
for result in bindings:
    activity_ent = result["activity"]["value"]
    # video_ent = result["video"]["value"]
    prev_object_name = result["prevObjectName"]["value"]
    action_ent = result["action"]["value"]
    action_name = action_ent.replace("http://kgrc4si.home.kg/virtualhome2kg/ontology/action/", "")
    object_ent = result["object"]["value"]
    object_name = remove_url_and_id(object_ent)
    target_object_ent = None
    target_object_name = None
    if "targetObject" in result:
        target_object_ent = result["targetObject"]["value"]
        target_object_name = remove_url_and_id(target_object_ent)
    
    answer = "The person will " + action_name
    if action_name == "sit":
        answer += " on"
    elif action_name == "walk":
        answer += " to"
    answer += " the " + object_name
    if target_object_ent is not None:
        if action_name == "put":
            answer += " on the " + target_object_name
        elif action_name == "pour":
            answer += " into the " + target_object_name
        else:
            answer += " to the " + target_object_name
    answer += "."
    
    # action_ent = result["action"]["value"]
    frame = result["frame"]["value"]
    video_results = get_video_base64(activity_ent)
    video_base64 = video_results["results"]["bindings"][0]["base64"]["value"]
    frame_rate = video_results["results"]["bindings"][0]["frameRate"]["value"]
    time = (float(frame))/float(frame_rate)


    # base64デコードしてバイナリデータを取得
    decoded_data = base64.b64decode(video_base64)

    # ファイルに書き込み
    with open("./prototype/tmp/decoded" + str(i) + ".mp4", "wb") as file:
        file.write(decoded_data)

    # Command to execute ffmpeg
    print(time)
    command = "ffmpeg -i ./prototype/tmp/decoded" + str(i) + ".mp4 -r 30 -ss " + str(time) + " -frames:v 1  ./prototype/test/" + str(i) + ".png"

    # Execute the command
    subprocess.run(command, shell=True)


    if i >= test_size:
         few_shot_samples.append({
            "activity_entity": activity_ent,
            "prev_object_name": prev_object_name, 
            "action": action_name,
            "object": object_name,
            "target_object": target_object_name,
            "frame": frame,
            "answer": answer
        })
    else:
        ground_truth.append({
            "activity_entity": activity_ent,
            "prev_object_name": prev_object_name, 
            "action": action_name,
            "object": object_name,
            "target_object": target_object_name,
            "frame": frame,
            "answer": answer
        })

    # Convert the ground_truth variable to a JSON list
    few_shot_samples_json = json.dumps(few_shot_samples)
    ground_truth_json = json.dumps(ground_truth)

    # Save the JSON list to a file
    with open('prototype/test/few_shot_samples.json', 'w') as file:
        file.write(few_shot_samples_json)

    with open('prototype/test/ground_truth.json', 'w') as file:
        file.write(ground_truth_json)

    i += 1

42.758620689655174


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino -

15.172413793103448
36.55172413793103


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino -

10.689655172413794
5.517241379310345


...

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino -

36.55172413793103


Output #0, image2, to './prototype/test/107.png':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf61.1.100
  Stream #0:0(und): Video: png, rgb24(pc, gbr/unknown/unknown, progressive), 640x480, q=2-31, 200 kb/s, 30 fps, 30 tbn (default)
      Metadata:
        handler_name    : VideoHandler
        vendor_id       : [0][0][0][0]
        encoder         : Lavc61.3.100 png
[image2 @ 0x15b013ba0] The specified filename './prototype/test/107.png' does not contain an image sequence pattern or a pattern is invalid.
[image2 @ 0x15b013ba0] Use a pattern such as %03d for an image sequence or use the -update option (with -frames:v 1 if needed) to write a single image.
[out#0/image2 @ 0x15b014e00] video:343KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: unknown
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.03 bitrate=N/A speed=0.268x    
ffmpeg version 7.0 Copyright (c) 2000

22.06896551724138
18.620689655172413


Output #0, image2, to './prototype/test/108.png':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf61.1.100
  Stream #0:0(und): Video: png, rgb24(pc, gbr/unknown/unknown, progressive), 640x480, q=2-31, 200 kb/s, 30 fps, 30 tbn (default)
      Metadata:
        handler_name    : VideoHandler
        vendor_id       : [0][0][0][0]
        encoder         : Lavc61.3.100 png
[image2 @ 0x136e041a0] The specified filename './prototype/test/108.png' does not contain an image sequence pattern or a pattern is invalid.
[image2 @ 0x136e041a0] Use a pattern such as %03d for an image sequence or use the -update option (with -frames:v 1 if needed) to write a single image.
[out#0/image2 @ 0x136e040d0] video:349KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: unknown
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.06 bitrate=N/A speed=0.742x    
ffmpeg version 7.0 Copyright (c) 2000